In [4]:
!pip install threading

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [26]:
# Import required libraries
import logging
import os
from dotenv import load_dotenv
from flask import Flask, request
from openai import OpenAI
from twilio.rest import Client
from twilio.twiml.voice_response import VoiceResponse, Gather
from flask_socketio import SocketIO, emit
import threading
from werkzeug.serving import make_server
from pyngrok import ngrok

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

True

In [27]:
print("Current environment variables:")
print(f"NGROK_AUTH_TOKEN: {'Found' if os.getenv('NGROK_AUTH_TOKEN') else 'Missing'}")
print(f"TWILIO_ACCOUNT_SID: {'Found' if os.getenv('TWILIO_ACCOUNT_SID') else 'Missing'}")
print(f"TWILIO_AUTH_TOKEN: {'Found' if os.getenv('TWILIO_AUTH_TOKEN') else 'Missing'}")
print(f"OPENAI_API_KEY: {'Found' if os.getenv('OPENAI_API_KEY') else 'Missing'}")

Current environment variables:
NGROK_AUTH_TOKEN: Found
TWILIO_ACCOUNT_SID: Found
TWILIO_AUTH_TOKEN: Found
OPENAI_API_KEY: Found


In [28]:
# Initialize Flask app
app = Flask(__name__)

# Initialize Twilio client
twilio_client = Client(os.getenv("TWILIO_ACCOUNT_SID"), os.getenv("TWILIO_AUTH_TOKEN"))

# Initialize OpenAI client
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize SocketIO
socketio = SocketIO(app, async_mode='threading')

# Initialize NGROK
ngrok_client = ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))



INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /home/vscode/.config/ngrok/ngrok


In [29]:
# Server thread class for running in notebook
class ServerThread(threading.Thread):
    def __init__(self, app):
        threading.Thread.__init__(self)
        self.srv = make_server('127.0.0.1', 5005, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        logger.info('Starting Server')
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

# Global server thread
server_thread = None

In [30]:
@app.route("/", methods=["GET"])
def home():
    """Test route to verify server is running"""
    return "Twilio Flask app is running!"

@app.route("/answer", methods=["POST"])
def answer_call():
    """Handle incoming phone calls"""
    logger.info("Received incoming call")
    response = VoiceResponse()

    # Add a greeting and wait for the caller to speak
    gather = Gather(
        input="speech", 
        action="/process_speech", 
        timeout=3, 
        speech_timeout="auto"
    )
    gather.say("Nine-one-one what is your emergency.")
    response.append(gather)

    # If the user doesn't say anything, try again
    response.redirect("/answer")

    return str(response)

In [18]:
@app.route("/call_results", methods=["GET"])
def call_results():
    """Display the results of the call processing"""
    # Get ui.html from the current directory
    with open("ui.html", "r") as f:
        return f.read()

@app.route("/test_socket", methods=["GET"])
def test_socket():
    """Test the websocket connection"""
    test_data = {
        "priority": "GREEN",
        "department": "POLICEDEPT",
        "summary": "This is a test message",
        "confidence": 95,
    }
    socketio.emit("send_message", test_data)
    return test_data

AssertionError: View function mapping is overwriting an existing endpoint function: call_results

In [19]:
@app.route("/process_speech", methods=["POST"])
def process_speech():
    """Process the speech input from the caller"""
    logger.info("Processing speech from call")
    # Get the speech input
    speech_result = request.values.get("SpeechResult", "")
    logger.info(f"Received speech: {speech_result}")

    if speech_result:
        # Process the text using the process call function
        result = process_call(speech_result)
        logger.info(f"Processing result: {result}")

        # Create a response
        response = VoiceResponse()
        response.say(f"I heard: {speech_result}")
        socketio.emit("send_message", result)
        response.say(f"Processing result: {result}")
    else:
        # If no speech was detected
        logger.warning("No speech detected")
        response = VoiceResponse()
        response.say("I'm sorry, I didn't catch that. Please try again.")
        response.redirect("/answer")

    return str(response)

AssertionError: View function mapping is overwriting an existing endpoint function: process_speech

In [20]:
def start_server():
    global server_thread
    if server_thread is None:
        server_thread = ServerThread(app)
        server_thread.start()
        socketio.init_app(app)
        
        # Start ngrok
        public_url = ngrok.connect(5005)
        print(f"\nServer is running!")
        print(f"Your public URL is: {public_url}")
        print("\nIMPORTANT: Go to your Twilio phone number configuration and set the Voice Webhook URL to:")
        print(f"{public_url}/answer")

def stop_server():
    global server_thread
    if server_thread is not None:
        server_thread.shutdown()
        ngrok.disconnect()
        server_thread = None
        print("Server stopped!")

In [21]:
# Start everything
start_server()

INFO:__main__:Starting Server
INFO:pyngrok.ngrok:Opening tunnel named: http-5005-2e3e0c31-86cc-4141-a431-1d5872a38b31


INFO:pyngrok.process.ngrok:t=2024-11-16T21:49:06+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2024-11-16T21:49:06+0000 lvl=info msg="using configuration at default config path" path=/home/vscode/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2024-11-16T21:49:06+0000 lvl=info msg="open config file" path=/home/vscode/.config/ngrok/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2024-11-16T21:49:06+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
ERROR:pyngrok.process.ngrok:t=2024-11-16T21:49:07+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"

ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERRO

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
# Stop the server
stop_server()

: 

In [32]:
ngrok.disconnect(public_url)

NameError: name 'public_url' is not defined